In [2]:
install.packages("randomForest")

Installing package into 'C:/Users/lopez/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'randomForest' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\lopez\AppData\Local\Temp\RtmpC81pUc\downloaded_packages


In [55]:
library(randomForest)
library(dplyr)
library(readr)
library(e1071)
library(caret)

In [69]:
df <- read_csv('https://raw.githubusercontent.com/gabichulas/ia-uncuyo-2024/refs/heads/main/tp7-ml/data/arbolado-mendoza-dataset-train.csv', show_col_types = FALSE)
test_df <- read_csv('https://raw.githubusercontent.com/gabichulas/ia-uncuyo-2024/refs/heads/main/tp7-ml/data/arbolado-mza-dataset-test.csv', show_col_types = FALSE)
val_df <- read_csv('https://raw.githubusercontent.com/gabichulas/ia-uncuyo-2024/refs/heads/main/tp7-ml/data/arbolado-mendoza-dataset-validation.csv', show_col_types = FALSE)

In [70]:
set_1 <- df[df$inclinacion_peligrosa == 1, ]
set_0 <- df[df$inclinacion_peligrosa == 0, ]

# Sobremuestrear la clase minoritaria
set_1_over <- set_1 %>% sample_n(nrow(set_0), replace = TRUE)
data <- rbind(set_1_over, set_0)
table(data$inclinacion_peligrosa)


    0     1 
22633 22633 

In [71]:
x <- data[, !(colnames(data) %in% c("inclinacion_peligrosa", "id", "area_seccion","seccion", "ultima_modificacion"))]
y <- factor(data$inclinacion_peligrosa)

In [73]:
model <- randomForest(x = x, y = y, ntree = 900, mtry = 2)
print(model)

predicciones_test <- predict(model, newdata = val_df)

# Obtener las etiquetas verdaderas del conjunto de prueba
etiquetas_reales_test <- val_df$inclinacion_peligrosa  # Reemplaza 'inclinacion_peligrosa' con el nombre de la columna de etiquetas

# Calcular la matriz de confusión
confusion_matrix_test <- table(Predicted = predicciones_test, Actual = etiquetas_reales_test)

# Calcular precisión, recall y F1-score
precision <- sum(diag(confusion_matrix_test)) / sum(confusion_matrix_test)
recall <- diag(confusion_matrix_test) / rowSums(confusion_matrix_test)
f1_score <- 2 * (precision * recall) / (precision + recall)

# Imprimir la matriz de confusión y las métricas
print(confusion_matrix_test)
cat("Precisión: ", precision, "\n")
cat("Recall: ", recall, "\n")
cat("F1-Score: ", f1_score, "\n")


Call:
 randomForest(x = x, y = y, ntree = 900, mtry = 2) 
               Type of random forest: classification
                     Number of trees: 900
No. of variables tried at each split: 2

        OOB estimate of  error rate: 12.53%
Confusion matrix:
      0     1 class.error
0 17217  5416   0.2392966
1   257 22376   0.0113551
         Actual
Predicted    0    1
        0 5559  646
        1  139   38
Precisión:  0.8769978 
Recall:  0.8958904 0.2146893 
F1-Score:  0.8863434 0.3449377 


In [65]:
predicciones <- predict(model, newdata = test_df)

submission <- data.frame(ID = test_df$id, inclinacion_peligrosa = predicciones)

write.csv(submission, file = "submission.csv", row.names = FALSE)

print("Predicciones guardadas en submission.csv")

[1] "Predicciones guardadas en submission.csv"
